In [ ]:
!pip install pip --upgrade
!pip3 install newspaper3k
!apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!git clone https://github.com/lovit/textrank.git

In [ ]:
from newspaper import Article
import re

def crawling(url):
    # url = input('url을 입력하세요')
    article = Article(url,laguage='ko')
    article.download()
    article.parse()
    title = article.title
    text = article.text
    text = '.'.join(article.text.split('.')[:-2:])
    pattern = '[\n]' #추가코드 #특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text) #추가코드 #특수기호제거
    # text = ''.join(text.split('기자) ')[1]) # split 할 경우
    # text = re.search(r'기자\)\s*(.*)', text)[1] # 정규표현식으로 할경우
    return title , text

https://www.hani.co.kr/arti/culture/entertainment/816895.html

In [ ]:
from konlpy.tag import Komoran

news_title, news_text = crawling("https://www.hani.co.kr/arti/culture/entertainment/816895.html")
raw_lines = news_text.split('.')
article = ""
for line in raw_lines:
  pattern = '^[ ]'# 특수기호제거
  line = re.sub(pattern=pattern, repl='', string=line)
  article = article + line + "\n"

komoran = Komoran()
def komoran_tokenize(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

T_article = "" #본문 변수 초기화
for i in range(len(raw_lines)):
  line = komoran_tokenize(raw_lines[i]) #n번째 줄 토큰화
  sentence = "" #문장 변수 초기화
  for j in line:
    sentence = sentence + j + " " #토큰화 텍스트 합치기
  T_article = T_article + sentence + "\n" #본문 변수에 합친 토큰화 문장 넣기

sents = T_article.split('\n')
texts = article.split('\n')

In [ ]:
from textrank.textrank.summarizer import KeywordSummarizer
from textrank.textrank.summarizer import KeysentenceSummarizer

def komoran_tokenize(sent):
    words = sent.split()
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

keyword_extractor = KeywordSummarizer(
    tokenize = komoran_tokenize,
    window = -1,
    verbose = False
)
summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenize,
    min_sim = 0.5,
    verbose = True
)
keysents = summarizer.summarize(sents)
i = 0
for idx, rank, komoran_sent in keysents:
    print('#{} ({:.3}) : {}'.format(idx, rank, texts[idx]), end='\n\n')

calculating textrank sentence similarity was done with 10 sents
trained TextRank. n sentences = 10
#2 (1.32) : 배우 송혜교(35), 송중기(32)는 이날 오후 하객 300명이 참석한 가운데 서울 신라호텔 영빈관에서 결혼식을 올렸다

#0 (1.13) : 31일 결혼식을 올린 ‘송송커플’과 중국배우 장쯔이

#7 (1.03) : 두 사람은 신혼여행을 바로 떠나지는 않았으며, 송중기가 지난 1월 마련한 서울 용산구 이태원의 단독 주택에 신혼살림을 차릴 것으로 전해졌다

#5 (0.862) : 두 사람은 지난해 폭발적인 인기를 모은 드라마 <태양의 후예>에 함께 출연하며 사랑을 키웠고, 지난 7월 결혼 소식을 발표했다

#3 (0.848) : 가수 옥주현이 축가를 불렀고, 송중기의 ‘절친’인 배우 이광수가 신랑·신부의 미래를 축복하며 편지를 낭송했다

#1 (0.811) : 태양의 후예>(KBS)가 빚어낸 ‘송송커플’이 31일 화촉을 밝혔다

#9 (0.15) : 

#8 (0.15) : 이주현 기자 edigna@hani

#6 (0.15) : 이들은 지난해 연말 ‘KBS 연기대상’ 대상을 공동수상하기도 했다

#4 (0.15) : 결혼식은 외부 비공개로 진행됐지만 예식 장소가 야외라 결혼식을 보려는 국내외 팬들이 장사진을 이뤘다

